In [17]:
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import tensorflow as tf
import sys
import random
import gensim.downloader as api

In [2]:
vocab_size = 22000
batch_size = 8
eps = 0.0001
k=20

Load the original (biased) embedding

For details of this model, see:

https://fasttext.cc/docs/en/english-vectors.html<br>
https://arxiv.org/abs/1712.09405<br>
https://arxiv.org/abs/1607.01759<br>

In [ ]:
wiki_model = api.load("fasttext-wiki-news-subwords-300")

[========================--------------------------] 49.1% 470.2/958.4MB downloaded

`cvs` refers to the "context vectors" or "output vector" produced during training -- see "Distributed Representations of Words and Phrases and their Compositionality" Mikolov et al

In [38]:
wvs = wiki_model.wv.vectors[:vocab_size]
cvs = wiki_model.trainables.syn1neg[:vocab_size]

Gender word pairs

In [4]:
gender_word_pairs_raw = [('he','she'),('man','woman'),('his','her'),('himself','herself'), ('him','her'),('men','women'),('husband','wife'),('boy','girl'),('men','women'),('brother','sister'),('mother','father'),('aunt','uncle'),('grandfather','grandmother'),('son','daughter'),('waiter','waitress'),('niece','nephew'),('girlfriend','boyfriend'),('father-in-law','mother-in-law'),('great uncle','great aunt'),('mr','ms'),('god','goddess'),('stepbrother','stepsister'),('stepmother','stepfather'),('son-in-law','daughter-in-law'),('brother-in-law','sister-in-law'),('landlord','landlady'),('hero','heroine'),('king','queen'),('duke','duchess'),('actor','actress'),('prince','princess'),('host','hostess'),('papa','mama'),('wizard','witch'),('monk','nun')]


Filter out words that are not in the first `vocab_size` vectors of the embedding (vectors are sorted by word frequency in the original corpus)

In [8]:
gender_word_pairs = []
for gw1,gw2 in gender_word_pairs_raw:
    try:
        i = wiki_model.wv.vocab[gw1].index
    except:
        print('{} not in vocab'.format(gw1))
        continue;
    try:
        j = wiki_model.wv.vocab[gw2].index
    except:
        print('{} not in vocab'.format(gw))
        continue;
    if j > vocab_size:
        print('{} has index {}, too large for vocab size {}'.format(gw1,j,vocab_size))
    elif i > vocab_size:
        print('{} has index {}, too large for vocab size {}'.format(gw2,j,vocab_size))
    else:
        gender_word_pairs.append((gw1,gw2))

father-in-law not in vocab
great uncle not in vocab
stepbrother has index 71120, too large for vocab size 22000
stepfather has index 21353, too large for vocab size 22000
son-in-law not in vocab
brother-in-law not in vocab
landlord has index 39383, too large for vocab size 22000
host has index 22230, too large for vocab size 22000


In [49]:
np.save('gender_word_pairs', gender_word_pairs)

In [9]:
male, female = zip(*gender_word_pairs)

In [10]:
male_words = list(male)
female_words = list(female)

In [11]:
male_inds = np.array([wiki_model.wv.vocab[w].index for w in male_words])

female_inds = np.array([wiki_model.wv.vocab[w].index for w in female_words])



From Bolukbasi, Tolga et all "Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings"

https://github.com/tolga-b/debiaswe/blob/master/data/gender_specific_full.json

In [24]:
appropriately_gendered_words = ["he", "his", "her", "she", "him", "man", "women", "men", "woman", "spokesman", "wife", "himself", "son", "mother", "father", "chairman",
"daughter", "husband", "guy", "girls", "girl", "boy", "boys", "brother", "spokeswoman", "female", "sister", "male", "herself", "brothers", "dad",
"actress", "mom", "sons", "girlfriend", "daughters", "lady", "boyfriend", "sisters", "mothers", "king", "businessman", "grandmother",
"grandfather", "deer", "ladies", "uncle", "males", "congressman", "grandson", "bull", "queen", "businessmen", "wives", "widow",
"nephew", "bride", "females", "aunt", "prostate cancer", "lesbian", "chairwoman", "fathers", "moms", "maiden", "granddaughter",
"younger brother", "lads", "lion", "gentleman", "fraternity", "bachelor", "niece", "bulls", "husbands", "prince", "colt", "salesman", "hers",
"dude", "beard", "filly", "princess", "lesbians", "councilman", "actresses", "gentlemen", "stepfather", "monks", "ex girlfriend", "lad",
"sperm", "testosterone", "nephews", "maid", "daddy", "mare", "fiance", "fiancee", "kings", "dads", "waitress", "maternal", "heroine",
"nieces", "girlfriends", "sir", "stud", "mistress", "lions", "estranged wife", "womb", "grandma", "maternity", "estrogen", "ex boyfriend",
"widows", "gelding", "diva", "teenage girls", "nuns", "czar", "ovarian cancer", "countrymen", "teenage girl", "penis", "bloke", "nun",
"brides", "housewife", "spokesmen", "suitors", "menopause", "monastery", "motherhood", "brethren", "stepmother", "prostate",
"hostess", "twin brother", "schoolboy", "brotherhood", "fillies", "stepson", "congresswoman", "uncles", "witch", "monk", "viagra",
"paternity", "suitor", "sorority", "macho", "businesswoman", "eldest son", "gal", "statesman", "schoolgirl", "fathered", "goddess",
"hubby", "stepdaughter", "blokes", "dudes", "strongman", "uterus", "grandsons", "studs", "mama", "godfather", "hens", "hen", "mommy",
"estranged husband", "elder brother", "boyhood", "baritone", "grandmothers", "grandpa", "boyfriends", "feminism", "countryman",
"stallion", "heiress", "queens", "witches", "aunts", "semen", "fella", "granddaughters", "chap", "widower", "salesmen", "convent",
"vagina", "beau", "beards", "handyman", "twin sister", "maids", "gals", "housewives", "horsemen", "obstetrics", "fatherhood",
"councilwoman", "princes", "matriarch", "colts", "ma", "fraternities", "pa", "fellas", "councilmen", "dowry", "barbershop", "fraternal",
"ballerina"]

In [27]:
def filter_app_gen_indicies(words):
    def get_index(w):
        try: 
            return wiki_model.wv.vocab[w].index
        except:
            print("not in model: {}".format(w))
            return vocab_size+10
    indices = list(map(lambda word: get_index(word), words))
    indices = list(filter(lambda i: i < vocab_size, indices))
    return np.flip(np.sort(indices))

In [28]:
app_gen_indices = get_indices_in_matrix(appropriately_gendered_words)

not in model: prostate cancer
not in model: younger brother
not in model: ex girlfriend
not in model: estranged wife
not in model: ex boyfriend
not in model: teenage girls
not in model: ovarian cancer
not in model: teenage girl
not in model: twin brother
not in model: eldest son
not in model: hubby
not in model: estranged husband
not in model: elder brother
not in model: twin sister


In [29]:
inapp_gen_indices = np.array(list((filter(lambda x: x not in app_gen_indices, np.arange(vocab_size)))))
inapp_gen_indices

array([    0,     1,     2, ..., 21997, 21998, 21999])

Per Mikolov's paper "Distributed Representations of Words and Phrases and their Compositionality," we use the following definition of log conditional probability.

$$ \log P(w_O|w_I) \approx \log \sigma ({v'_{wo}}^T v_{wI}) + \sum_{i=1}^{k} [\log {\sigma ({{-v'_{wi}}^T v_{wI}})}] $$

https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf

Per the paper, we use the unigram distribution raised to the 3/4th power from which to draw samples

In [39]:
word_counts = np.array([wiki_model.wv.vocab[wiki_model.wv.index2word[i]].count for i in range(vocab_size)])
word_counts_power = np.power(word_counts,.75)
sampling_dist = np.true_divide(word_counts_power,np.sum(word_counts_power))

Generate batch produces indicies of words to debias and indicies of the k samples with which to calculate the probability

In [40]:
# generate batch data
def generate_batch(batch_size):
    indicies_wv = np.random.choice(inapp_gen_indices,batch_size,replace=False)
    sampling_dist_inapp_gen_indices = sampling_dist[inapp_gen_indices] / np.sum(sampling_dist[inapp_gen_indices])
    samples = np.random.choice(inapp_gen_indices,size=k,replace=False,p=sampling_dist_inapp_gen_indices)
    samples_2d = np.array([samples for i in range(batch_size)])
    return indicies_wv, samples_2d

In [37]:
indices_wv, samples_2d = generate_batch(batch_size)

In [24]:
original_weights = wiki_model.wv.vectors[:vocab_size,:]
original_cv_weights = cvs[:vocab_size,:]

In [56]:
graph = tf.Graph()

with graph.as_default():
    with tf.name_scope('inputs'):
        indices_samples = tf.placeholder(tf.int32, shape=[batch_size])
        samples_inputs = tf.placeholder(tf.int32, shape=[batch_size,k])
    with tf.device('/cpu:0'):
        with tf.name_scope('embeddings'):
            # initialize the the biased embedding weights
            embeddings_wv = tf.Variable(wvs)
            embeddings_cv = tf.Variable(cvs)
        # calculate log P(w_i|m_j) for all w_i in indices_samples and all m_j in male_words
        embed_wv = tf.nn.embedding_lookup(embeddings_wv,indices_samples)
        embed_cv_he = tf.nn.embedding_lookup(embeddings_wv,male_inds)
        embed_sv = tf.nn.embedding_lookup(embeddings_wv,samples_inputs)
        prod_he = tf.matmul(embed_wv, tf.transpose(embed_cv_he))
        first_term_he = tf.math.log_sigmoid(prod_he)
        embed_sv = tf.reshape(embed_sv,[vector_dim,batch_size,k])
        prod_2_he = tf.tensordot(embed_cv_he, -1*embed_sv, axes=[[1],[0]])
        prod_2_log_sig_he = tf.math.log_sigmoid(prod_2_he)
        second_term_he = tf.reduce_sum(prod_2_log_sig_he,2)
        prob_he = first_term_he + tf.transpose(second_term_he)
        
        # calculate log P(w_i|f_j) for all w_i in indices_samples and all m_j in female_words
        embed_cv_she = tf.nn.embedding_lookup(embeddings_wv,female_inds)
        prod_she = tf.matmul(embed_wv, tf.transpose(embed_cv_she))
        first_term_she = tf.math.log_sigmoid(prod_she)
        prod_2_she = tf.tensordot(embed_cv_she, -1*embed_sv, axes=[[1],[0]])
        prod_2_log_sig_she = tf.math.log_sigmoid(prod_2_she)
        second_term_she = tf.reduce_sum(prod_2_log_sig_she,2)
        prob_she = first_term_she + tf.transpose(second_term_she)


    with tf.name_scope('loss'):
        loss = tf.reduce_sum(tf.abs(prob_he - prob_she))
    
    tf.summary.scalar('loss', loss)
    
    with tf.name_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(0.003).minimize(loss)
    
    merged = tf.summary.merge_all()
    
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()     

Test `generate_batch`

In [57]:
indices_wv, samples = generate_batch(batch_size)

In [58]:
feed_dict = {indices_samples: indices_wv, samples_inputs : samples}
with tf.Session(graph=graph) as sess:
    init.run()
    res = sess.run(loss, feed_dict)
    print(res)

105905.45


In [59]:
num_steps = 1000001

with tf.Session(graph=graph) as session:

    init.run()
    print('Initialized')
    min_loss = sys.maxsize
    average_loss = 0
    for step in range(num_steps):
        indices_wv, samples = generate_batch(batch_size)
        feed_dict = {indices_samples: indices_wv, samples_inputs : samples}



      # Define metadata variable.
        run_metadata = tf.RunMetadata()

        _, summary, loss_val = session.run([optimizer, merged, loss],
                                         feed_dict=feed_dict,
                                         run_metadata=run_metadata)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
        # The average loss is an estimate of the loss over the last 2000
        # batches.
            print('Average loss at step ', step, ': ', average_loss)
            if average_loss < min_loss:
                print('New min loss, saving embedding')
                wvs_embed_trained = embeddings_wv.eval()

                cvs_embed_trained = embeddings_cv.eval()

                min_loss = average_loss
                
            average_loss = 0    
            
    # Save the model for checkpoints.
    saver.save(session, './model_0')
    

Initialized
('Average loss at step ', 0, ': ', 145808.46875)
New min loss, saving embedding
('Average loss at step ', 2000, ': ', 42407.46003369141)
New min loss, saving embedding
('Average loss at step ', 4000, ': ', 5409.074129760742)
New min loss, saving embedding
('Average loss at step ', 6000, ': ', 3871.2420838012695)
New min loss, saving embedding
('Average loss at step ', 8000, ': ', 3066.7472525939943)
New min loss, saving embedding
('Average loss at step ', 10000, ': ', 2479.593877166748)
New min loss, saving embedding
('Average loss at step ', 12000, ': ', 2069.0306828308107)
New min loss, saving embedding
('Average loss at step ', 14000, ': ', 1737.6057858886718)
New min loss, saving embedding
('Average loss at step ', 16000, ': ', 1505.0508660736084)
New min loss, saving embedding
('Average loss at step ', 18000, ': ', 1315.2425707702637)
New min loss, saving embedding
('Average loss at step ', 20000, ': ', 1191.3702390136718)
New min loss, saving embedding
('Average loss 

('Average loss at step ', 190000, ': ', 237.15282377624513)
New min loss, saving embedding
('Average loss at step ', 192000, ': ', 235.380104927063)
New min loss, saving embedding
('Average loss at step ', 194000, ': ', 234.71038902282714)
New min loss, saving embedding
('Average loss at step ', 196000, ': ', 230.02637878417968)
New min loss, saving embedding
('Average loss at step ', 198000, ': ', 229.08726819610595)
New min loss, saving embedding
('Average loss at step ', 200000, ': ', 226.67460459136962)
New min loss, saving embedding
('Average loss at step ', 202000, ': ', 225.1324543838501)
New min loss, saving embedding
('Average loss at step ', 204000, ': ', 223.38908647155762)
New min loss, saving embedding
('Average loss at step ', 206000, ': ', 223.0223366241455)
New min loss, saving embedding
('Average loss at step ', 208000, ': ', 221.05493197631836)
New min loss, saving embedding
('Average loss at step ', 210000, ': ', 217.70255585479737)
New min loss, saving embedding
('A

('Average loss at step ', 392000, ': ', 129.4457576522827)
('Average loss at step ', 394000, ': ', 125.52518164825439)
New min loss, saving embedding
('Average loss at step ', 396000, ': ', 129.25232378387452)
('Average loss at step ', 398000, ': ', 128.47520518493653)
('Average loss at step ', 400000, ': ', 124.71199565505981)
New min loss, saving embedding
('Average loss at step ', 402000, ': ', 125.79988800811768)
('Average loss at step ', 404000, ': ', 127.81772748565673)
('Average loss at step ', 406000, ': ', 122.22835911941529)
New min loss, saving embedding
('Average loss at step ', 408000, ': ', 123.12983395385743)
('Average loss at step ', 410000, ': ', 124.14998905181885)
('Average loss at step ', 412000, ': ', 124.14651356124878)
('Average loss at step ', 414000, ': ', 124.02596559143066)
('Average loss at step ', 416000, ': ', 122.56485943984985)
('Average loss at step ', 418000, ': ', 120.54446464538574)
New min loss, saving embedding
('Average loss at step ', 420000, ': 

KeyboardInterrupt: 

In [44]:
np.save('fasttext_wiki_debias_prob_wvs_weights', wvs_embed_trained)


In [61]:
np.save('fasttext_wiki_debias_prob_cvs_weights', cvs_embed_trained)


In [45]:
wvs_embed_trained_norm = np.sqrt(np.sum(np.square(wvs_embed_trained), 1, keepdims=True))
wvs_embed_trained_normalized_embeddings = wvs_embed_trained / wvs_embed_trained_norm


In [46]:
np.save('fasttext_wiki_debias_prob_wvs', wvs_embed_trained_normalized_embeddings)


Save model as .bin to faciliate comparison

In [47]:
def convert_np_to_bin_model(np_vectors,model_name):
    with open(model_name+'.txt', 'w') as we:
        we.write('{} {}\n'.format(vocab_size,vector_dim))
        for i in range(vocab_size):
            w = wiki_model.wv.index2word[i]
            vec = np_vectors[i]
            we.write('{} '.format(w))
            for v in vec:
                we.write(str(v) + ' ')
            we.write('\n')
    model = KeyedVectors.load_word2vec_format(model_name+'.txt', binary=False)
    model.save_word2vec_format(model_name+'.bin', binary=True)
    print('created model '+model_name+'.bin')

In [48]:
convert_np_to_bin_model(wvs_embed_trained_normalized_embeddings,'fasttext_wiki_debias_prob')


created model fasttext_wiki_debias_prob.bin
